In [1]:
import numpy as np
import pandas as pd
import datetime as dt

from itertools import islice
import os

import klcalculator

In [2]:
print("Start Time:", dt.datetime.now())

Start Time: 2019-07-01 13:14:08.771259


In [3]:
# The following functions generate all possible sub-lists (subsequences) from an input list (main sequence)
# csub generates continuous sequences.
# ncsub generates non-continuous sequences.
# sublists combines them.

# function to generate all the sub lists 
def csub(list1): 

    # store all the sublists  
    sublist = [[]] 

    # first loop  
    for i in range(len(list1) + 1): 

        # second loop  
        for j in range(i + 1, len(list1) + 1): 
              
            # slice the subarray  
            sub = list1[i:j] 
            sublist.append(sub) 

    return sublist

def ncsub(seq, s=0):
    if seq:
        x = seq[:1]
        xs = seq[1:]
        p2 = s % 2
        p1 = not p2
        return [x + ys for ys in ncsub(xs, s + p1)] + ncsub(xs, s + p2)
    else:
        return [[]] if s >= 3 else []
    
def sublists(x):
    return csub(x) + ncsub(x)

In [4]:
# Load the Inmate Admissions trajectory dataset (can also be downloaded from the NYC Open Data portal)
# Static variables: RACE and GENDER
# Trajectory variable: INMATE_STATUS_CODE

inmate = pd.read_csv('Inmate_Admissions.csv', header=0)
pd.set_option('display.expand_frame_repr', True)

inmate['ADMITTED_DT'] = inmate['ADMITTED_DT'].apply(lambda x: dt.datetime.strptime(x, '%m/%d/%Y %H:%M:%S %p'))

inmate = inmate.sort_values(['INMATEID', 'ADMITTED_DT'])

inmate.head(10)

,INMATEID,ADMITTED_DT,DISCHARGED_DT,RACE,GENDER,INMATE_STATUS_CODE,TOP_CHARGE
164452,17,2017-02-23 11:18:00,02/28/2017 03:01:03 PM,UNKNOWN,F,DE,NaN
129218,30,2016-01-09 01:04:23,01/14/2016 06:13:04 AM,BLACK,M,CS,155.25
152112,30,2016-03-08 08:42:00,03/24/2016 09:23:03 PM,BLACK,M,DE,NaN
6270,36,2018-02-14 08:11:13,NaN,UNKNOWN,M,DE,125.25
166978,40,2015-10-01 05:16:00,10/01/2015 09:24:05 PM,BLACK,M,DE,NaN
236106,49,2017-06-07 01:33:42,06/09/2017 04:21:03 AM,BLACK,M,DE,NaN
103036,52,2014-12-29 04:01:00,01/03/2015 12:10:05 AM,BLACK,M,DE,NaN
206966,52,2017-10-21 12:21:00,10/24/2017 03:40:01 AM,BLACK,M,DE,NaN
39271,52,2018-10-24 01:27:00,NaN,BLACK,M,SSR,110-120.10
131884,56,2014-08-05 10:41:00,05/11/2015 12:05:04 AM,BLACK,M,SSR,NaN


In [5]:
inmate_ddup = inmate.drop_duplicates(['INMATEID'], keep='last')

inmate_static = inmate_ddup[['INMATEID', 'RACE', 'GENDER']].copy()

inmate_static.head(5) # inmate_static dataframe now holds the static varibles (race and gender), only one row per inmate.

,INMATEID,RACE,GENDER
164452,17,UNKNOWN,F
152112,30,BLACK,M
6270,36,UNKNOWN,M
166978,40,BLACK,M
236106,49,BLACK,M


In [6]:
inmate_filtered = inmate[['INMATEID', 'ADMITTED_DT', 'INMATE_STATUS_CODE']].copy()

inmate_filtered.sort_values(by=['INMATEID', 'ADMITTED_DT'], inplace=True)

inmate_filtered.head(5) # inmate_filtered holds the trajectory variable INMATE_STATE_CODE.

# Next we need to put the INMATE_STATUS_CODEs of each inmate into one single row.

,INMATEID,ADMITTED_DT,INMATE_STATUS_CODE
164452,17,2017-02-23 11:18:00,DE
129218,30,2016-01-09 01:04:23,CS
152112,30,2016-03-08 08:42:00,DE
6270,36,2018-02-14 08:11:13,DE
166978,40,2015-10-01 05:16:00,DE


In [7]:
df=inmate_filtered['INMATEID'] \
    .value_counts() \
    .to_frame().reset_index() \
    .rename(columns={'index':'INM_ID', 'INMATEID':'count'})
df2=df.sort_values(by=['INM_ID'])

df2.head(5)

,INM_ID,count
79736,17,1
51026,30,2
76027,36,1
74821,40,1
93521,49,1


In [8]:
# Input list initialization
Input = inmate_filtered['INMATE_STATUS_CODE'].tolist()
  
# list of length in which we have to split 
length_to_split = df2['count']
  
# Using islice 
Inputt = iter(Input) 
Output = [list(islice(Inputt, elem)) 
          for elem in length_to_split] 

# Input list initialization
Input2 = inmate_filtered['INMATEID'].tolist()
  
# Using islice 
Inputt = iter(Input2) 
Output2 = [list(islice(Inputt, elem)) 
          for elem in length_to_split]

In [9]:
id_list = [x+1 for x in range(len(Output))]
                   
df6 = pd.DataFrame(list(zip(id_list, [x[0] for x in Output2], Output)), columns=['id', 'inmate_id', 'seq'])

df6["seq_n_steps"] = df6['seq'].str.len()

df6.head(25) # df6 now contains the sequence of INMATE_STATUS_CODE, one row per inmate

,id,inmate_id,seq,seq_n_steps
0,1,17,[DE],1
1,2,30,"[CS, DE]",2
2,3,36,[DE],1
3,4,40,[DE],1
4,5,49,[DE],1
5,6,52,"[DE, DE, SSR]",3
6,7,56,"[SSR, DE]",2
7,8,57,[SSR],1
8,9,58,"[DE, SSR]",2
9,10,63,[DPV],1


In [10]:
inmate_subseq = []
inmate_id = []
j = 0

for x in Output[:1000]: # Only pick up the first 1000 records
    x1 = x[:15] # limit to max 15 steps (significant reduction of computation time)
    y = sublists(x1) # use set to remove duplicates
    tot = tuple(tuple(z) for z in y) # change list to (immutable) tuple for use as dictionary keys
    d1 = dict.fromkeys(tot) # remove duplicates
    lol = list(list(z) for z in d1) # change back to list
    inmate_subseq.append(lol)
    inmate_id.append([df6['inmate_id'][j]] * len(lol))
    j += 1

flat_list = [item for sublist in inmate_subseq for item in sublist]
flat_list_id = [item for sublist in inmate_id for item in sublist]

df5 = pd.DataFrame(list(zip(flat_list_id, flat_list)), columns=['inmate_id','subseq'])

df5["n_steps"] = df5['subseq'].str.len()

df5["subseq_str"] = ['>'.join(map(str, l)) for l in df5['subseq']]

df5 = df5.loc[(df5['n_steps'] > 0) & (df5['n_steps'] <= 3)]
df5["val"] = 1

# df5 now holds all possible subsequences (subseq_str) inmates can have.
# Note that one inmate can have more than one row if there are more than one subsequence associated with that inmate.

# Create a pivot table for df5 (one inmate per row, all subsequences across the columns)
tbl = pd.pivot_table(df5, values='val', index=['inmate_id'], columns=['subseq_str'], aggfunc=np.sum, fill_value=0)

tbl.head(10)

subseq_str,CS,CS>CS,CS>CS>CS,CS>CS>CSP,CS>CS>DE,CS>CS>DEP,CS>CS>DPV,CS>CS>SCO,CS>CS>SSR,CS>CSP,...,SSR>SCO>DPV,SSR>SCO>SCO,SSR>SCO>SSR,SSR>SSR,SSR>SSR>CS,SSR>SSR>DE,SSR>SSR>DEP,SSR>SSR>DPV,SSR>SSR>SCO,SSR>SSR>SSR
inmate_id,,,,,,,,,,,,,,,,,,,,,
17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
56,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Merge the static and trajectory data together
merged_inner = pd.merge(left=inmate_static,right=tbl, left_on='INMATEID', right_on='inmate_id')

merged_inner.head(10)

,INMATEID,RACE,GENDER,CS,CS>CS,CS>CS>CS,CS>CS>CSP,CS>CS>DE,CS>CS>DEP,CS>CS>DPV,...,SSR>SCO>DPV,SSR>SCO>SCO,SSR>SCO>SSR,SSR>SSR,SSR>SSR>CS,SSR>SSR>DE,SSR>SSR>DEP,SSR>SSR>DPV,SSR>SSR>SCO,SSR>SSR>SSR
0,17,UNKNOWN,F,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,30,BLACK,M,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,36,UNKNOWN,M,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40,BLACK,M,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,49,BLACK,M,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,52,BLACK,M,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,56,BLACK,M,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,57,BLACK,M,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,58,BLACK,M,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,63,BLACK,M,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
l = list(merged_inner)
l.pop(0) # remove the inmate id as it will not be used in risk calculation

features = l

data = merged_inner[features]
dataset = list(zip(*(data[fn].tolist() for fn in features)))

original_risks = klcalculator.find_risks_for_records(dataset)
klcalculator.display_risks(features, original_risks)

In [13]:
print(dt.datetime.now())

2019-07-01 13:14:36.846952
